<a href="https://colab.research.google.com/github/akash13s/ASL-Interpreter/blob/main/asl_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade -q accelerate bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install -q av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.8 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-zq9bqtjv
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-zq9bqtjv
  Resolved https://github.com/huggingface/transformers.git to commit 33eef992503689ba1af98090e26d3e98865b2a9b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 47.1 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.47.0.dev0-py3-none-any.whl size=10053251 sha256=dfde99ec26e70df8191a6b5a09c652f7b84205dceee89afab7ee93b40a4234bc
  Stored in directory: /tmp/pip-ephem-wheel-cache-o84hp7qp/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16


In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
import os, sys

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Computer Vision/CV Project/Code'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

sys.path.append(GOOGLE_DRIVE_PATH)

['raw_videos', 'valid_clips.csv', 'dataset.py', 'asl_to_text.ipynb']


In [8]:
from transformers import BitsAndBytesConfig, LlavaNextVideoForConditionalGeneration, LlavaNextVideoProcessor
import torch
import av
import numpy as np

from dataset import VideoDataset, getDefaultTransform
from torch.utils.data import DataLoader

## Load and Set Up the Model

Loading the model with quantization enabled

In [9]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

processor = LlavaNextVideoProcessor.from_pretrained("llava-hf/LLaVA-NeXT-Video-7B-hf")
model = LlavaNextVideoForConditionalGeneration.from_pretrained(
    "llava-hf/LLaVA-NeXT-Video-7B-hf",
    quantization_config=quantization_config,
    device_map='auto'
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/741 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

## Process and Generate Using the Model

Here we will apply the prompt, and generate outputs

In [11]:
def process_and_generate(model, processor, frames, prompt_text):
  # Prepare conversation template
  conversation = [
    {
      "role": "user",
      "content": [
        {"type": "text", "text": prompt_text},
        {"type": "video"},
      ],
    },
  ]

  # Apply chat template
  prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

  # Process inputs for the model
  inputs = processor([prompt], videos=[frames], return_tensors="pt").to(model.device)

  # Set generation parameters
  generate_kwargs = {"max_new_tokens": 100, "do_sample": True, "top_p": 0.9}

  # Generate response
  output = model.generate(**inputs, **generate_kwargs)
  generated_text = processor.batch_decode(output, skip_special_tokens=True)

  return generated_text

## Iterate Through the DataLoader for Batch Processing

Now we will go through each batch in our dataloader, sample frames, and pass them through the model.


In [82]:
def sample_frames(frames, num_sample_frames):
    # Generate indices for uniform sampling
    indices = torch.linspace(0, frames.shape[1] - 1, steps=num_sample_frames).long()

    # Sample frames using the calculated indices
    sampled_frames = frames[:, indices, :, :, :]  # Tensor of frames: (batch_size, num_sample_frames, C, H, W)

    # Check if the entire sampled batch is zero-padded
    if torch.all(sampled_frames == 0):
        print("Skipping zero-padded frames due to missing or invalid video data.")
        return None

    return sampled_frames

In [83]:
video_dir = os.path.join(GOOGLE_DRIVE_PATH, 'data/raw_videos')
csv_file = os.path.join(GOOGLE_DRIVE_PATH, 'data/valid_clips.csv')

# Try several prompts for each clip
prompts = ["Translate the ASL gestures in this video into English text.",
            "Provide a detailed translation of the ASL signing in this video.",
            "Convert the sign language in this video to text."]

transform = getDefaultTransform()

dataset = VideoDataset(video_dir=video_dir, csv_file=csv_file, transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0)
num_sample_frames=32

# Loop over each batch in the dataloader
for batch in dataloader:
    # Get frames and sentences from batch
    frames_batch, sentences_batch = batch  # `frames_batch` has shape [batch_size, num_frames, C, H, W]

    # Loop over each clip in the batch
    for i in range(len(frames_batch)):
        # Extract frames and sentence for the individual clip
        frames = frames_batch[i].unsqueeze(0)  # Shape: [1, num_frames, C, H, W]
        sentence = sentences_batch[i]

        # Sample frames from each video in the batch
        frames = sample_frames(frames, num_sample_frames)

        # Generate response from the model for this clip and prompt
        generated_text = process_and_generate(model, processor, frames, prompts[2])

        print("\nGenerated Text:", generated_text)
        print("Ground Truth Sentence:", sentence)

ValueError: Could not make batched video from [tensor([[[[[0.3882, 0.4000, 0.4078,  ..., 0.5020, 0.5176, 0.5294],
           [0.3961, 0.4078, 0.4157,  ..., 0.5059, 0.5216, 0.5333],
           [0.4039, 0.4118, 0.4157,  ..., 0.5176, 0.5255, 0.5373],
           ...,
           [0.2745, 0.2627, 0.2627,  ..., 0.5020, 0.5020, 0.4980],
           [0.2745, 0.2627, 0.2627,  ..., 0.5098, 0.5020, 0.5059],
           [0.2706, 0.2549, 0.2745,  ..., 0.5098, 0.5059, 0.5059]],

          [[0.6510, 0.6471, 0.6431,  ..., 0.7137, 0.7176, 0.7255],
           [0.6588, 0.6549, 0.6549,  ..., 0.7176, 0.7255, 0.7333],
           [0.6627, 0.6627, 0.6627,  ..., 0.7216, 0.7255, 0.7373],
           ...,
           [0.5216, 0.5137, 0.5098,  ..., 0.7176, 0.7137, 0.7137],
           [0.5255, 0.5137, 0.5137,  ..., 0.7176, 0.7098, 0.7098],
           [0.5216, 0.5059, 0.5294,  ..., 0.7216, 0.7098, 0.7098]],

          [[0.4549, 0.4549, 0.4549,  ..., 0.5255, 0.5294, 0.5490],
           [0.4627, 0.4667, 0.4667,  ..., 0.52

In [53]:
def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.

    Args:
        container (av.container.input.InputContainer): PyAV container.
        indices (List[int]): List of frame indices to decode.

    Returns:
        np.ndarray: np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

In [81]:
video_path = os.path.join(GOOGLE_DRIVE_PATH, 'data/raw_videos/--7E2sU6zP4_11-5-rgb_front.mp4')

container = av.open(video_path)

# sample uniformly 8 frames from the video (we can sample more for longer videos)
total_frames = container.streams.video[0].frames
indices = np.arange(0, total_frames, total_frames / 32).astype(int)
clip_baby = read_video_pyav(container, indices)

# Each "content" is a list of dicts and you can add image/video/text modalities
conversation = [
      {
          "role": "user",
          "content": [
              {"type": "text", "text": "What do you see in this video"},
              {"type": "video"},
              ],
      },
]

prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

inputs = processor([prompt], videos=[clip_baby], padding=True, return_tensors="pt").to(model.device)
generate_kwargs = {"max_new_tokens": 100, "do_sample": True, "top_p": 0.9}

output = model.generate(**inputs, **generate_kwargs)
generated_text = processor.batch_decode(output, skip_special_tokens=True)

print(generated_text)

['USER: \nWhat do you see in this video ASSISTANT: In**\n.']
